Will do PCA and maybe some subset selection in this file to determine the size of the latent space in the autoencoder mechanism in USAD 

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as stats
from scipy.stats import pearsonr, spearmanr, mode
from IPython.display import display, HTML
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
test = pd.read_csv("path to csv")
train = pd.read_csv("path to csv")
#combine the csvs

In [ ]:
df = pd.concat([test, train], ignore_index=True)
df = df.sort_values(by='Timestamp')

X = df.sample(n=50, random_state=42)
y = X["Normal/Attack"]
X = X.drop(["Timestamp" , "Normal/Attack", "PASS_ID"], axis = 1)


df = df.drop(["Timestamp" , "Normal/Attack", "PASS_ID" ] , axis = 1)

In [ ]:
display(df)

In [ ]:
na_counts = len(df) - df.isna().sum()
na_df = pd.DataFrame({'Column Name': na_counts.index, 'Number of Non NAs': na_counts.values})

display(na_df)

In [ ]:
df_normal = StandardScaler().fit_transform(df)

In [ ]:
pca = PCA()
principalComponents = pca.fit_transform(df_normal)

# Calculate the cumulative explained variance
explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Find the number of components for 90% variance
n_over_90 = np.argmax(explained_variance >= 0.95) + 1 # "+1" because np.argmax returns zero-based indices
print(f"Number of components to explain 90% of the variance: {n_over_90}")

# Plot cumulative explained variance to visualize
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.plot(explained_variance)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance by Components')
plt.grid(True)
plt.show()

In [ ]:
explained_variance_by_components = [(i+1, v) for i, v in enumerate(explained_variance)]
for item in explained_variance_by_components:
    print(f"Number of components: {item[0]}, Total explained variance: {item[1]:.4f}")

We also do some recursive feature elimintation with l1 penalty 

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', penalty = "l1", max_iter = 10000)
rfe = RFE(model)
fit = rfe.fit(X, y)

print("Number of features selected by subset selection to explain most of the variance: ")
print(sum(fit.support_))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def forward_selection(X, y):
    remaining_features = list(X.columns)
    selected_features = []
    while remaining_features:
        scores = []
        for feature in remaining_features:
            temp_features = selected_features + [feature]
            model = LogisticRegression(max_iter=10000)
            score = cross_val_score(model, X[temp_features], y, cv=5).mean()
            scores.append((score, feature))
        scores.sort()  # ascending order, so the best score is the last one
        best_score, best_feature = scores[-1]
        remaining_features.remove(best_feature)
        selected_features.append(best_feature)
        print(f'Added feature {best_feature} with score {best_score}')
    return selected_features

selected_features = forward_selection(X, y)
print(f"Number of features selected by forward subset selection: {len(selected_features)}")
print(f"Selected features: {selected_features}")

